In [13]:
import pandas as pd
import statsmodels.api as sm
from dmba import stepwise_selection, AIC_score
from sklearn.linear_model import LinearRegression

In [3]:
data = pd.read_csv("./datasets/insurance.csv")
data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [4]:
cleaned_data = pd.get_dummies(data, drop_first=True)

cleaned_data

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,1,0,0,1
1,18,33.770,1,1725.55230,1,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,1,0
3,33,22.705,0,21984.47061,1,0,1,0,0
4,32,28.880,0,3866.85520,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0,1,0,0
1334,18,31.920,0,2205.98080,0,0,0,0,0
1335,18,36.850,0,1629.83350,0,0,0,1,0
1336,21,25.800,0,2007.94500,0,0,0,0,1


In [5]:
cleaned_data = cleaned_data.rename(columns={'sex_male':'sex','smoker_yes':'smoker'})

In [6]:
cleaned_data.to_csv("./datasets/insurance_cleaned.csv", index=False)

In [7]:
outcome = 'charges'
predictors = ['age','bmi','children','smoker']

insurance_full_lm = sm.OLS(cleaned_data[outcome],cleaned_data[predictors])
results = insurance_full_lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                charges   R-squared (uncentered):                   0.872
Model:                            OLS   Adj. R-squared (uncentered):              0.872
Method:                 Least Squares   F-statistic:                              2277.
Date:                Wed, 31 Aug 2022   Prob (F-statistic):                        0.00
Time:                        19:04:45   Log-Likelihood:                         -13629.
No. Observations:                1338   AIC:                                  2.727e+04
Df Residuals:                    1334   BIC:                                  2.729e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
age          197.6732     11.589     17.058      0.000     174.939     220.407
bmi           28.0102     15.948      1.756      0.079      -3.276      59.296
children     240.5490    144.750      1.662      0.097     -43.414     524.512
smoker      2.331e+04    433.801     53.732      0.000    2.25e+04    2.42e+04
==============================================================================
Omnibus:                      277.987   Durbin-Watson:                   2.070
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              640.617
Skew:                           1.140   Prob(JB):                    7.79e-140
Kurtosis:                       5.509   Cond. No.                         126.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Cross validation
Se refiere a validar nuestro model en base a una parte de los datos (muestra) que no se usa para el entrenamiento.

El algoritmo más común es:
- K-fold cross validation

Se utiliza cuando hay muchos registros y también cuando existe incertidumbre.

Reto o actividad: Implementar k-fold

## Model selection y stepwise regression


In [8]:
car_data = pd.read_csv("./datasets/carprice.csv")
car_data = car_data.set_index("car_ID")

predictors = [
    #'car_ID', 
    'symboling', 
    #'CarName', 
    'fueltype', 
    'aspiration',
    'doornumber', 
    'carbody', 
    'drivewheel', 
    'enginelocation', 
    'wheelbase',
    'carlength', 
    'carwidth', 
    'carheight', 
    'curbweight', 
    'enginetype',
    'cylindernumber', 
    'enginesize', 
    'fuelsystem', 
    'boreratio', 
    'stroke',
    'compressionratio', 
    'horsepower',
    'peakrpm', 
    'citympg', 
    'highwaympg'
]

outcome = "price"


car_data['cylindernumber']=car_data['cylindernumber'].replace({'four':4,'six':6,'five':5,'three':4,'twelve':12,'two':2,'eight':8})
car_data['doornumber']=car_data['doornumber'].replace({'four':4,'two':2})

#car_data['enginetype']=car_data['enginetype'].replace({'dohc':1, 'ohcv':2, 'ohc':3, 'l':4, 'rotor':5, 'ohcf':6, 'dohcv':7})

#car_data['fuelsystem']=car_data['fuelsystem'].replace({'mpfi':1, '2bbl':2, 'mfi':3, '1bbl':4, 'spfi':5, '4bbl':6, 'idi':7, 'spdi':8})

#car_data['carbody']=car_data['carbody'].replace({'convertible':1, 'hatchback':2, 'sedan':3, 'wagon':4, 'hardtop':5})

# Reemplazar los valores
cleaned_car_data = pd.get_dummies(car_data[predictors], drop_first=True)
cleaned_car_data[outcome] = car_data[outcome]
cleaned_car_data = cleaned_car_data.rename(columns={"fueltype_gas":"fueltype","aspiration_turbo":"turbo"})
#cleaned_car_data
cleaned_car_data

,symboling,doornumber,wheelbase,carlength,carwidth,carheight,curbweight,cylindernumber,enginesize,boreratio,...,enginetype_ohcv,enginetype_rotor,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi,price
car_ID,,,,,,,,,,,,,,,,,,,,,
1,3,2,88.6,168.8,64.1,48.8,2548,4,130,3.47,...,0,0,0,0,0,0,1,0,0,13495.0
2,3,2,88.6,168.8,64.1,48.8,2548,4,130,3.47,...,0,0,0,0,0,0,1,0,0,16500.0
3,1,2,94.5,171.2,65.5,52.4,2823,6,152,2.68,...,1,0,0,0,0,0,1,0,0,16500.0
4,2,4,99.8,176.6,66.2,54.3,2337,4,109,3.19,...,0,0,0,0,0,0,1,0,0,13950.0
5,2,4,99.4,176.6,66.4,54.3,2824,5,136,3.19,...,0,0,0,0,0,0,1,0,0,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,-1,4,109.1,188.8,68.9,55.5,2952,4,141,3.78,...,0,0,0,0,0,0,1,0,0,16845.0
202,-1,4,109.1,188.8,68.8,55.5,3049,4,141,3.78,...,0,0,0,0,0,0,1,0,0,19045.0
203,-1,4,109.1,188.8,68.9,55.5,3012,6,173,3.58,...,1,0,0,0,0,0,1,0,0,21485.0


In [9]:
cleaned_car_data.columns

Index(['symboling', 'doornumber', 'wheelbase', 'carlength', 'carwidth',
       'carheight', 'curbweight', 'cylindernumber', 'enginesize', 'boreratio',
       'stroke', 'compressionratio', 'horsepower', 'peakrpm', 'citympg',
       'highwaympg', 'fueltype', 'turbo', 'carbody_hardtop',
       'carbody_hatchback', 'carbody_sedan', 'carbody_wagon', 'drivewheel_fwd',
       'drivewheel_rwd', 'enginelocation_rear', 'enginetype_dohcv',
       'enginetype_l', 'enginetype_ohc', 'enginetype_ohcf', 'enginetype_ohcv',
       'enginetype_rotor', 'fuelsystem_2bbl', 'fuelsystem_4bbl',
       'fuelsystem_idi', 'fuelsystem_mfi', 'fuelsystem_mpfi',
       'fuelsystem_spdi', 'fuelsystem_spfi', 'price'],
      dtype='object')

In [10]:
predictors = cleaned_car_data.columns
predictors = predictors.drop("price")
outcome = "price"

car_lm = sm.OLS(cleaned_car_data[outcome],cleaned_car_data[predictors])

results = car_lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.910
Method:                 Least Squares   F-statistic:                     56.45
Date:                Wed, 31 Aug 2022   Prob (F-statistic):           1.82e-76
Time:                        19:04:45   Log-Likelihood:                -1865.6
No. Observations:                 205   AIC:                             3807.
Df Residuals:                     167   BIC:                             3934.
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
symboling             344.4416    249.763      1.379      0.170    -148.658     837.542
doornumber            153.2709    315.604      0.486      0.628    -469.817     776.358
wheelbase              94.6250    104.395      0.906      0.366    -111.480     300.730
carlength             -47.0405     52.770     -0.891      0.374    -151.223      57.142
carwidth              612.8443    232.329      2.638      0.009     154.164    1071.524
carheight             270.2873    135.872      1.989      0.048       2.039     538.536
curbweight              4.0771      1.817      2.244      0.026       0.490       7.664
cylindernumber       1186.2134    691.166      1.716      0.088    -178.335    2550.762
enginesize            125.9415     27.158      4.637      0.000      72.324     179.559
boreratio           -2497.4948   1684.021     -1.483      0.140   -5822.208     827.218
stroke              -3894.3189    934.698     -4.166      0.000   -5739.666   -2048.972
compressionratio    -1743.6042    468.846     -3.719      0.000   -2669.234    -817.974
horsepower             -8.3569     21.755     -0.384      0.701     -51.307      34.593
peakrpm                 2.5687      0.652      3.939      0.000       1.281       3.856
citympg                16.0084    156.795      0.102      0.919    -293.548     325.565
highwaympg            175.7831    145.507      1.208      0.229    -111.487     463.053
fueltype            -5.658e+04   1.53e+04     -3.689      0.000   -8.69e+04   -2.63e+04
turbo                1707.6294    930.054      1.836      0.068    -128.550    3543.808
carbody_hardtop     -3330.1510   1484.881     -2.243      0.026   -6261.708    -398.594
carbody_hatchback   -3750.4083   1295.438     -2.895      0.004   -6307.953   -1192.863
carbody_sedan       -3073.8122   1409.180     -2.181      0.031   -5855.916    -291.708
carbody_wagon       -4680.6058   1541.244     -3.037      0.003   -7723.440   -1637.772
drivewheel_fwd        367.2714   1125.353      0.326      0.745   -1854.481    2589.023
drivewheel_rwd       2192.5205   1250.712      1.753      0.081    -276.724    4661.765
enginelocation_rear  9837.9989   2579.111      3.814      0.000    4746.134    1.49e+04
enginetype_dohcv     2452.6330   3839.539      0.639      0.524   -5127.658       1e+04
enginetype_l        -2371.0764   1420.827     -1.669      0.097   -5176.175     434.022
enginetype_ohc       2159.4529    956.243      2.258      0.025     271.570    4047.336
enginetype_ohcf      1430.8316   1699.345      0.842      0.401   -1924.135    4785.798
enginetype_ohcv     -4069.1829   1282.699     -3.172      0.002   -6601.577   -1536.788
enginetype_rotor     1.338e+04   2949.600      4.537      0.000    7559.165    1.92e+04
fuelsystem_2bbl       256.0067    963.071      0.266      0.791   -1645.357    2157.371
fue

In [14]:
y = cleaned_car_data[outcome]
X = cleaned_car_data[predictors]

In [17]:
def train_model(variables):
    if len(variables)==0:
        return None
    
    model = LinearRegression()
    model.fit(X[variables],y)
    return model

def score_model(model, variables):
    if len(variables)==0:
        return AIC_score(y,[y.mean()]*len(y), model, df=1)
    
    return AIC_score(y, model.predict(X[variables]), model)


best_model, best_variables = stepwise_selection(X.columns,train_model,score_model,verbose=True)

Variables: symboling, doornumber, wheelbase, carlength, carwidth, carheight, curbweight, cylindernumber, enginesize, boreratio, stroke, compressionratio, horsepower, peakrpm, citympg, highwaympg, fueltype, turbo, carbody_hardtop, carbody_hatchback, carbody_sedan, carbody_wagon, drivewheel_fwd, drivewheel_rwd, enginelocation_rear, enginetype_dohcv, enginetype_l, enginetype_ohc, enginetype_ohcf, enginetype_ohcv, enginetype_rotor, fuelsystem_2bbl, fuelsystem_4bbl, fuelsystem_idi, fuelsystem_mfi, fuelsystem_mpfi, fuelsystem_spdi, fuelsystem_spfi
Start: score=4268.94, constant
Step: score=3974.82, add enginesize
Step: score=3948.19, add drivewheel_rwd
Step: score=3927.72, add enginelocation_rear
Step: score=3879.82, add carwidth
Step: score=3871.14, add peakrpm
Step: score=3867.03, add enginetype_ohcv
Step: score=3860.03, add stroke
Step: score=3849.06, add boreratio
Step: score=3836.56, add enginetype_rotor
Step: score=3828.80, add turbo
Step: score=3820.20, add enginetype_ohc
Step: score=

In [18]:
best_variables

['enginesize',
 'drivewheel_rwd',
 'enginelocation_rear',
 'carwidth',
 'peakrpm',
 'enginetype_ohcv',
 'stroke',
 'boreratio',
 'enginetype_rotor',
 'turbo',
 'enginetype_ohc',
 'carbody_hatchback',
 'fuelsystem_spdi',
 'carbody_hardtop',
 'carbody_wagon',
 'carbody_sedan',
 'carheight',
 'enginetype_ohcf',
 'enginetype_dohcv']

In [19]:
best_model_stats = sm.OLS(cleaned_car_data[outcome],cleaned_car_data[best_variables])

results = best_model_stats.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.974
Model:                            OLS   Adj. R-squared (uncentered):              0.971
Method:                 Least Squares   F-statistic:                              365.8
Date:                Wed, 31 Aug 2022   Prob (F-statistic):                   2.71e-136
Time:                        19:17:15   Log-Likelihood:                         -1894.8
No. Observations:                 205   AIC:                                      3828.
Df Residuals:                     186   BIC:                                      3891.
Df Model:                          19                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
enginesize            198.0054      9.867     20.068      0.000     178.540     217.471
drivewheel_rwd       1928.1551    646.194      2.984      0.003     653.344    3202.967
enginelocation_rear  8003.4700   2178.048      3.675      0.000    3706.618    1.23e+04
carwidth              153.0289    119.127      1.285      0.201     -81.985     388.043
peakrpm                 1.1579      0.450      2.571      0.011       0.269       2.047
enginetype_ohcv     -4760.5481   1141.113     -4.172      0.000   -7011.736   -2509.360
stroke              -5033.6682    811.217     -6.205      0.000   -6634.037   -3433.299
boreratio           -5349.1112   1171.042     -4.568      0.000   -7659.342   -3038.880
enginetype_rotor     1.183e+04   1669.249      7.089      0.000    8540.703    1.51e+04
turbo                3701.7733    568.683      6.509      0.000    2579.875    4823.672
enginetype_ohc       2210.7364    665.584      3.321      0.001     897.672    3523.800
carbody_hatchback   -4001.3861   1212.947     -3.299      0.001   -6394.287   -1608.485
fuelsystem_spdi     -2633.7428   1081.446     -2.435      0.016   -4767.219    -500.267
carbody_hardtop     -4240.4726   1468.198     -2.888      0.004   -7136.933   -1344.012
carbody_wagon       -3517.5003   1364.306     -2.578      0.011   -6209.003    -825.997
carbody_sedan       -2732.3091   1212.079     -2.254      0.025   -5123.500    -341.119
carheight             122.9787    109.971      1.118      0.265     -93.971     339.929
enginetype_ohcf       489.4042   1311.069      0.373      0.709   -2097.074    3075.882
enginetype_dohcv     6076.4741   2864.383      2.121      0.035     425.620    1.17e+04
==============================================================================
Omnibus:                       30.256   Durbin-Watson:                   1.489
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              202.812
Skew:                           0.122   Prob(JB):                     9.12e-45
Kurtosis:                       7.867   Cond. No.                     8.29e+04
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 8.29e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
# Retirar fuel_system, 